In [418]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

from pycaret.classification import * 

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

In [419]:
#Loading Dataset
dataset = pd.read_parquet('dataframes_blog/dataset_for_model_final.parquet') 

In [420]:
dataset = dataset[dataset['drop'] == 0]

In [421]:
# Get Train Test Split
df_train = dataset[dataset['cum_week'] <= dataset["train_split"]].dropna()
df_test = dataset[dataset['cum_week'] > dataset["train_split"]].dropna()

In [422]:
X_train = df_train[extended_features]
y_train = df_train[injured_pred]

X_test = df_test[extended_features]
y_test = df_test[injured_pred]

In [423]:
#Loading Models
week_model = load_model('model_1_week_tuned')
month_model = load_model('model_1_month_tuned')
quarter_model = load_model('model_1_quarter_tuned')
semester_model = load_model('model_1_semester_tuned')
year_model = load_model('model_1_year_tuned')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [424]:
# Cleaning NaNs

dataset_no_nans = dataset.fillna(0)

In [425]:
# Choosing the features we are going to use

dataset_to_predict = dataset_no_nans[['Height', 'Weight', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'age', 'cum_injury_total', \
    'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum', 'Sh_cum', 'SoT_cum', 'CrdY_cum', 'CrdR_cum', \
    'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum', 'SCA_cum', 'GCA_cum', 'Cmp_cum', 'Att_cum', \
    'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum', 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', \
    'PKwon_cum', 'PKcon_cum', 'Serie A_cum', 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', \
    'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum', 'Copa América_cum', 'Away_cum', 'Home_cum', \
    'Neutral_cum']]

In [426]:
#Using saved models to predict injuries

predicted_injuries_1_week = week_model[23].predict(dataset_to_predict)
predicted_injuries_1_month = month_model[23].predict(dataset_to_predict)
predicted_injuries_1_quarter = quarter_model[23].predict(dataset_to_predict)
predicted_injuries_1_semester = semester_model[23].predict(dataset_to_predict)
predicted_injuries_1_year = year_model[23].predict(dataset_to_predict)

In [520]:
# Assigning the complete y target to y_target variable

y_target = list(y_train).append(list(y_test))

In [521]:
# Creating predictions_df to include the whole dataset plus predicted injuries and actual injures

predictions_df = dataset_to_predict
predictions_df['predicted_injuries_1_week'] = predicted_injuries_1_week
predictions_df['predicted_injuries_1_month'] = predicted_injuries_1_month
predictions_df['predicted_injuries_1_quarter'] = predicted_injuries_1_quarter
predictions_df['predicted_injuries_1_semester'] = predicted_injuries_1_semester
predictions_df['predicted_injuries_1_year'] = predicted_injuries_1_year

predictions_df['actual_injuries'] = y_target

In [522]:
predictions_df

,Height,Weight,defender,attacker,midfielder,goalkeeper,right_foot,age,cum_injury_total,weeks_since_last_injury,Min_cum,Gls_cum,Ast_cum,PK_cum,PKatt_cum,Sh_cum,SoT_cum,CrdY_cum,CrdR_cum,Touches_cum,Press_cum,Tkl_cum,Int_cum,Blocks_cum,xG_cum,npxG_cum,xA_cum,SCA_cum,GCA_cum,Cmp_cum,Att_cum,Prog_cum,Carries_cum,Prog.1_cum,Succ_cum,Att.1_cum,Fls_cum,Fld_cum,Off_cum,Crs_cum,TklW_cum,OG_cum,PKwon_cum,PKcon_cum,Serie A_cum,Premier League_cum,La Liga_cum,Ligue 1_cum,Bundesliga_cum,Champions Lg_cum,Europa Lg_cum,FIFA World Cup_cum,UEFA Nations League_cum,UEFA Euro_cum,Copa América_cum,Away_cum,Home_cum,Neutral_cum,predicted_injuries_1_week,predicted_injuries_1_month,predicted_injuries_1_quarter,predicted_injuries_1_semester,predicted_injuries_1_year,actual_injuries
1,184.0,80.0,0,0,1,0,1,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,None
2,184.0,80.0,0,0,1,0,1,18.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,None
3,184.0,80.0,0,0,1,0,1,18.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,None
4,184.0,80.0,0,0,1,0,1,18.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,None
5,184.0,80.0,0,0,1,0,1,18.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910250,171.0,60.0,0,1,0,0,1,19.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
1910251,171.0,60.0,0,1,0,0,1,19.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
1910252,171.0,60.0,0,1,0,0,1,20.0,1.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.1,1.0,0.0,3.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,None
1910253,171.0,60.0,0,1,0,0,1,20.0,1.0,13.0,82.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,16.0,10.0,0.0,0.0,0.0,0.2,0.2,0.1,1.0,0.0,8.0,10.0,0.0,13.0,6.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,None


In [523]:
# Merging the complete dataset with the predictions of the model

dataset_with_prediction = pd.merge(dataset, predictions_df[['predicted_injuries_1_week', 'predicted_injuries_1_month', \
    'predicted_injuries_1_quarter', 'predicted_injuries_1_semester', 'predicted_injuries_1_year']], left_index=True, right_index=True)

In [524]:
# Picking the columns we are going to keep

dataset_with_prediction = dataset_with_prediction[['FBRefID', 'name', 'date', 'Min', 'Position:', 'cum_week', 'defender', 'attacker', 'midfielder', 'goalkeeper', \
    'cum_injury_in_1_week', 'cum_injury_in_4_week',  'cum_injury_in_12_week', 'cum_injury_in_26_week', 'cum_injury_in_52_week', 'Min_cum', \
    'predicted_injuries_1_week', 'predicted_injuries_1_month', 'predicted_injuries_1_quarter', 'predicted_injuries_1_semester', \
    'predicted_injuries_1_year']]

In [525]:
# Creating unique_predicted_injuries to filter out repeated injuries

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 1) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 1), 'unique_predicted_injuries_1_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 1) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 0), 'unique_predicted_injuries_1_week'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 0) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 0), 'unique_predicted_injuries'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 0) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 1), 'unique_predicted_injuries_1_week'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_month'] == 1) & (dataset_with_prediction['predicted_injuries_1_month']. \
    shift(1) == 1), 'unique_predicted_injuries_1_month'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_month'] == 1) & (dataset_with_prediction['predicted_injuries_1_month']. \
    shift(1) == 0), 'unique_predicted_injuries_1_month'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_month'] == 0) & (dataset_with_prediction['predicted_injuries_1_month']. \
    shift(1) == 0), 'unique_predicted_injuries_1_month'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_month'] == 0) & (dataset_with_prediction['predicted_injuries_1_month']. \
    shift(1) == 1), 'unique_predicted_injuries_1_month'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_quarter'] == 1) & (dataset_with_prediction['predicted_injuries_1_quarter']. \
    shift(1) == 1), 'unique_predicted_injuries_1_quarter'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_quarter'] == 1) & (dataset_with_prediction['predicted_injuries_1_quarter']. \
    shift(1) == 0), 'unique_predicted_injuries_1_quarter'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_quarter'] == 0) & (dataset_with_prediction['predicted_injuries_1_quarter']. \
    shift(1) == 0), 'unique_predicted_injuries_1_quarter'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_quarter'] == 0) & (dataset_with_prediction['predicted_injuries_1_quarter']. \
    shift(1) == 1), 'unique_predicted_injuries_1_quarter'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_semester'] == 1) & (dataset_with_prediction['predicted_injuries_1_semester']. \
    shift(1) == 1), 'unique_predicted_injuries_1_semester'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_semester'] == 1) & (dataset_with_prediction['predicted_injuries_1_semester']. \
    shift(1) == 0), 'unique_predicted_injuries_1_semester'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_semester'] == 0) & (dataset_with_prediction['predicted_injuries_1_semester']. \
    shift(1) == 0), 'unique_predicted_injuries_1_semester'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_semester'] == 0) & (dataset_with_prediction['predicted_injuries_1_semester']. \
    shift(1) == 1), 'unique_predicted_injuries_1_semester'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_year'] == 1) & (dataset_with_prediction['predicted_injuries_1_year']. \
    shift(1) == 1), 'unique_predicted_injuries_1_year'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_year'] == 1) & (dataset_with_prediction['predicted_injuries_1_year']. \
    shift(1) == 0), 'unique_predicted_injuries_1_year'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_year'] == 0) & (dataset_with_prediction['predicted_injuries_1_year']. \
    shift(1) == 0), 'unique_predicted_injuries_1_year'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_year'] == 0) & (dataset_with_prediction['predicted_injuries_1_year']. \
    shift(1) == 1), 'unique_predicted_injuries_1_year'] = 0

In [526]:
# Getting the accumulated sum of predicted injuries by player

dataset_with_prediction['cum_predicted_injuries_1_week'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_1_week'].cumsum()
dataset_with_prediction['cum_predicted_injuries_1_month'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_1_month'].cumsum()
dataset_with_prediction['cum_predicted_injuries_1_quarter'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_1_quarter'].cumsum()
dataset_with_prediction['cum_predicted_injuries_1_semester'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_1_semester'].cumsum()
dataset_with_prediction['cum_predicted_injuries_1_year'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_1_year'].cumsum()

In [527]:
#Filtering the dataset for a subset of players

player_for_df_final = ['Lionel Messi', 'Cristiano Ronaldo', 'Neymar', 'Robert Lewandowski', 'Mohamed Salah', 'Kevin De Bruyne', 'Erling Haaland', \
'Harry Kane', 'Virgil van Dijk', 'Eden Hazard', 'Karim Benzema', 'Paul Pogba', 'Sergio Ramos', 'Jadon Sancho', 'Romelu Lukaku', \
'Joshua Kimmich', 'Antoine Griezmann', 'Toni Kroos', 'Jan Oblak', 'Manuel Neuer', 'Gareth Bale', 'Raheem Sterling', 'Frenkie de Jong', \
'Thiago Silva', 'Paulo Dybala']

small_dataset_with_prediction = pd.DataFrame([])

for n in player_for_df_final:
    small_dataset_with_prediction = small_dataset_with_prediction.append(dataset_with_prediction[dataset_with_prediction['name'] == n])

In [528]:
# Picking player to forecast

picked_player_df = small_dataset_with_prediction[small_dataset_with_prediction['name'] == 'Neymar']

In [537]:
# Function to Forecast players injuries and plot a chart

def forecast_it(weeks, df1):
    df = df1.reset_index()
    final_index = df.index.max()
    pos = 'cum_injury_in_'+str(weeks)+'_week'
    for n in range(weeks,0,-1): 
        df.loc[final_index-n+1, pos] = ''
    df.loc[df[pos] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = df[pos].shift(1) 
    df.loc[df['single_injury_prediction_in_'+str(weeks)+'_week'] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = \
        df['unique_predicted_injuries']
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['single_injury_prediction_in_'+str(weeks)+'_week'].cumsum()
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_prediction_in_'+str(weeks)+'_week'].fillna('')
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_in_'+str(weeks)+'_week']
    
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'type_for_'+str(weeks)] = 'Actual Injuries'
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] == "", 'type_for_'+str(weeks)] = 'Predicted Injuries'

    extra_point = df[df['type_for_'+str(weeks)] == 'Actual Injuries'].tail(1)
    extra_point['type_for_'+str(weeks)] = 'Predicted Injuries'
    df = pd.concat([df, extra_point]) #.reset_index(drop=True)

    df.loc[df['attacker'] == 1, 'position'] = 'attacker'
    df.loc[df['defender'] == 1, 'position'] = 'defender'
    df.loc[df['goalkeeper'] == 1, 'position'] = 'goalkeeper'
    df.loc[df['midfielder'] == 1, 'position'] = 'midfielder'

    df = df[['FBRefID', 'name', 'date', 'cum_week', 'position', 'Min_cum', 'cum_injury_prediction_in_'+str(weeks)+'_week', 'type_for_'+str(weeks)]]
    
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_prediction_in_'+str(weeks)+'_week'].ffill(axis=0)
    
    return extra_point

In [530]:
final_small_df_week = forecast_it(1, picked_player_df)
final_small_df_month = forecast_it(4, picked_player_df)
final_small_df_quarter = forecast_it(12, picked_player_df)
final_small_df_semester = forecast_it(26, picked_player_df)
final_small_df_year = forecast_it(52, picked_player_df)

In [547]:
final_small_df_month.tail(10)

,FBRefID,name,date,cum_week,position,Min_cum,cum_injury_prediction_in_4_week,type_for_4
389,69384e5d,Neymar,2021-10-10,432.0,attacker,1450.0,97.0,Actual Injuries
390,69384e5d,Neymar,2021-10-17,433.0,attacker,0.0,97.0,Actual Injuries
391,69384e5d,Neymar,2021-10-17,433.0,attacker,90.0,97.0,Actual Injuries
392,69384e5d,Neymar,2021-10-24,434.0,attacker,254.0,97.0,Actual Injuries
393,69384e5d,Neymar,2021-10-31,435.0,attacker,434.0,97.0,Actual Injuries
394,69384e5d,Neymar,2021-11-07,436.0,attacker,794.0,97.0,Predicted Injuries
395,69384e5d,Neymar,2021-11-14,437.0,attacker,884.0,97.0,Predicted Injuries
396,69384e5d,Neymar,2021-11-21,438.0,attacker,1018.0,97.0,Predicted Injuries
397,69384e5d,Neymar,2021-11-28,439.0,attacker,1372.0,97.0,Predicted Injuries
393,69384e5d,Neymar,2021-10-31,435.0,attacker,434.0,97.0,Predicted Injuries


In [531]:
final_small_df = final_small_df_week

final_small_df['cum_injury_prediction_in_4_week'] = list(final_small_df_month['cum_injury_prediction_in_4_week'])
final_small_df['type_for_4'] = list(final_small_df_month['type_for_4'])

final_small_df['cum_injury_prediction_in_12_week'] = list(final_small_df_quarter['cum_injury_prediction_in_12_week'])
final_small_df['type_for_12'] = list(final_small_df_quarter['type_for_12'])

final_small_df['cum_injury_prediction_in_26_week'] = list(final_small_df_semester['cum_injury_prediction_in_26_week'])
final_small_df['type_for_26'] = list(final_small_df_semester['type_for_26'])

final_small_df['cum_injury_prediction_in_52_week'] = list(final_small_df_year['cum_injury_prediction_in_52_week'])
final_small_df['type_for_52'] = list(final_small_df_year['type_for_52'])

In [542]:
final_small_df.tail(20)

,FBRefID,name,date,cum_week,position,Min_cum,cum_injury_prediction_in_1_week,type_for_1,cum_injury_prediction_in_4_week,type_for_4,cum_injury_prediction_in_12_week,type_for_12,cum_injury_prediction_in_26_week,type_for_26,cum_injury_prediction_in_52_week,type_for_52
379,69384e5d,Neymar,2021-07-11,419.0,attacker,2004.0,92.0,Actual Injuries,94.0,Actual Injuries,96.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
380,69384e5d,Neymar,2021-07-18,420.0,attacker,2004.0,92.0,Actual Injuries,95.0,Actual Injuries,96.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
381,69384e5d,Neymar,2021-07-25,421.0,attacker,2004.0,93.0,Actual Injuries,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
382,69384e5d,Neymar,2021-08-01,422.0,attacker,0.0,94.0,Actual Injuries,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
383,69384e5d,Neymar,2021-08-29,426.0,attacker,130.0,96.0,Actual Injuries,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
384,69384e5d,Neymar,2021-09-05,427.0,attacker,220.0,96.0,Actual Injuries,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
385,69384e5d,Neymar,2021-09-12,428.0,attacker,310.0,96.0,Actual Injuries,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
386,69384e5d,Neymar,2021-09-19,429.0,attacker,670.0,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
387,69384e5d,Neymar,2021-09-26,430.0,attacker,1030.0,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries
388,69384e5d,Neymar,2021-10-03,431.0,attacker,1360.0,96.0,Actual Injuries,97.0,Actual Injuries,97.0,Predicted Injuries,97.0,Predicted Injuries,96.0,Predicted Injuries


In [532]:
#Plotting Chart
def plot_it(weeks, df):

    df_final = df[len(df)-25:].sort_index()

    domain_min = df_final['cum_injury_prediction_in_'+str(weeks)+'_week'].min()-5
    domain_max = df_final['cum_injury_prediction_in_'+str(weeks)+'_week'].max()+5

    chart = alt.Chart(df_final).mark_line().encode(x=alt.X('date', timeUnit='yearmonthdate'), \
        y=alt.Y('cum_injury_prediction_in_'+str(weeks)+'_week', \
        scale=alt.Scale(domain=[domain_min, domain_max])), color=alt.Color('type_for_'+str(weeks)), strokeDash=alt.condition(
        alt.datum.symbol == 'type_for_'+str(weeks), alt.value([5, 5]), alt.value([0]))). \
        properties(title = str(df_final['name'].iloc[0])+"'s Injury Prediction for "+str(weeks)+" weeks", width=800, height=300)

    return chart
    

In [533]:
final_small_df.loc[final_small_df['cum_injury_prediction_in_12_week'] == '', 'cum_injury_prediction_in_12_week'] = None
final_small_df.loc[final_small_df['cum_injury_prediction_in_26_week'] == '', 'cum_injury_prediction_in_26_week'] = None
final_small_df.loc[final_small_df['cum_injury_prediction_in_52_week'] == '', 'cum_injury_prediction_in_52_week'] = None

In [534]:
final_small_df['cum_injury_prediction_in_12_week'] = final_small_df['cum_injury_prediction_in_12_week'].ffill()
final_small_df['cum_injury_prediction_in_26_week'] = final_small_df['cum_injury_prediction_in_26_week'].ffill()
final_small_df['cum_injury_prediction_in_52_week'] = final_small_df['cum_injury_prediction_in_52_week'].ffill()

In [535]:
plot_it(12, final_small_df)

alt.Chart(...)